In [ ]:
from notebooks import *

In [ ]:
%%time
joypy.joyplot(
    # Perf: kind='kde' and kind='counts' are roughly the same
    # kind='kde',
    kind='counts', bins=80,
    fill=False,
    linecolor='k',
    linewidth=.5,
    xlabels=True,
    ylabels=False,
    # overlap=.001,
    # overlap=.01,
    # overlap=.1,
    # overlap=.5,
    # overlap=1,
    # XXX This gets _very slow_ with more lines
    #   - Unusable >100, and our spectro Sxx are more like ~600
    data=[
        stats.distributions.norm.rvs(mu, 10, 100)
        for mu in np.arange(50)
    ],
)
plt.show()

In [ ]:
# Here's what's required to turn an FT into a periodogram (power by freq)
#   - TODO Oops, x-axis is still off by 2x...
x = ...  # Real signal, e.g. audio waveform
sample_rate = 22050
f, Pxx = scipy.signal.periodogram(x, fs=sample_rate, scaling='spectrum')
plt.plot(f, Pxx); plt.show()
plt.plot((np.abs(np.fft.rfft(x)) / len(x))**2 * 2); plt.show()

In [ ]:
# Diff spectro vs. librosa melspectrogram
f, t, Sxx = np.abs(spectro(audio, nperseg=nperseg, overlap=overlap, detrend='constant', scaling='density', mode='complex'))**2
Sxx = Sxx * 1024**2 * 8  # Works with scaling='density'
display(Sxx.shape)
plt.hist(Sxx.flatten(), bins=100, log=True); plt.show()
Sxx = librosa.power_to_db(np.dot(mel_basis, Sxx)); display(Sxx.shape)
# plt.pcolormesh(Sxx); [s.set_visible(False) for s in plt.gca().spines.values()]; plt.show()

n_fft = nperseg
Sxx = librosa.feature.melspectrogram(x.astype(float), sample_rate, None, n_fft, int(n_fft*(1-overlap)), n_mels=n_mels, power=2)
display(Sxx.shape)
plt.hist(Sxx.flatten(), bins=100, log=True); plt.show()
Sxx = librosa.power_to_db(np.dot(mel_basis, Sxx)); display(Sxx.shape)
# plt.pcolormesh(Sxx); [s.set_visible(False) for s in plt.gca().spines.values()]; plt.show()

In [ ]:
# Diff mfcc vs. librosa mfcc
n_mfcc = 1024
plt_mfcc(x, mfcc=partial(mfcc, mels_div=2, n_mfcc=n_mfcc, std=True), show_audio=False, fancy=False)
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter(''))
plt.show()

M = librosa.feature.mfcc(x.astype(float), sample_rate, n_mfcc=n_mfcc)
M = (M - M.mean(axis=1)[:, np.newaxis]) / M.std(axis=1)[:, np.newaxis]  # Standardize
plt.pcolormesh(M)
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter(''))
plt.gca().tick_params(labelsize=8)
[s.set_visible(False) for s in plt.gca().spines.values()]
plt.show()

In [ ]:
import dask
import dask.bag
import dask.dataframe as dd

dask.set_options(scheduler='processes')

df = pd.DataFrame({'a':range(10,20), 'b':range(20,30)}); df

ddf = dd.from_pandas(df, npartitions=8)
ddf.apply?

dask.bag.from_sequence(range(100)).compute()

In [ ]:
from datatypes import Recording
from load import *

recs_paths = load_recs_paths(['recordings-new'])
row = recs_paths.iloc[0]
rec = load_rec(row.dataset, row.path)

import dataclasses
dir(dataclasses)

# load_recs_data(recs_paths)

dataclasses.asdict(rec)
dataclasses.fields(rec)

(
  Field(name='dataset',type=<class 'str'>,default=<dataclasses._MISSING_TYPE object at 0x12621d4e0>,default_factory=<dataclasses._MISSING_TYPE object at 0x12621d4e0>,init=True,repr=True,hash=None,compare=True,metadata={}),
  Field(name='name',type=<class 'str'>,default=<dataclasses._MISSING_TYPE object at 0x12621d4e0>,default_factory=<dataclasses._MISSING_TYPE object at 0x12621d4e0>,init=True,repr=True,hash=None,compare=True,metadata={}),
  Field(name='species',type=<class 'str'>,default=<dataclasses._MISSING_TYPE object at 0x12621d4e0>,default_factory=<dataclasses._MISSING_TYPE object at 0x12621d4e0>,init=True,repr=True,hash=None,compare=True,metadata={}),
  Field(name='species_query',type=<class 'str'>,default=<dataclasses._MISSING_TYPE object at 0x12621d4e0>,default_factory=<dataclasses._MISSING_TYPE object at 0x12621d4e0>,init=True,repr=True,hash=None,compare=True,metadata={}),
  Field(name='basename',type=<class 'str'>,default=<dataclasses._MISSING_TYPE object at 0x12621d4e0>,de

In [ ]:
from potoo.plot import *

# repr(ggplot(mpg) + geom_bar(aes(x='class')))

theme_figsize('inline_short').rcParams

figsize('inline_short');
# figsize('inline_short', dpi=72);
# figsize('inline_short', dpi=150);

%time plt.plot(range(10)); display(plot_to_img())
%time plt.plot(range(10)); plt.show()

In [ ]:
from cache import *

@cache(version=4)
def foo(x): return x+2

foo(3)
foo(3, __cache_version=5)

In [ ]:
import pickle
import zict

f = zict.Func(pickle.dumps, pickle.loads, zict.File('/tmp/zict-0/'))
d = zict.LRU(
    n=2,
    d=f,
    on_evict=lambda k, v: print('evict: %s:%s' % (k, v)),
    weight=lambda k, v: 1,
)

In [ ]:
import pickle
import zict
from zict_util import PersistentBuffer

a = {}
b = zict.Func(pickle.dumps, pickle.loads, zict.File('/tmp/zict-0/'))
d = PersistentBuffer(
    fast=a,
    slow=b,
    n=2,
    weight=lambda k, v: 1,
    evict_fast_callbacks=[lambda k, v: print('evict_fast: %s:%s' % (k, v))],
    load_fast_callbacks=[lambda k, v: print('load_fast: %s:%s' % (k, v))],
)

show = lambda: print('fast[%s], slow[%s]' % (dict(d.fast), dict(d.slow)))

In [ ]:
show()
d['x'] = 1; show()
d['y'] = 2; show()
d['z'] = 3; show()
d['y']; show()
d['x']; show()
d['x'] = -1; show()
d['z']; show()
d['x'] = 1; show()
d['y'] = -2; show()
d['y']; show()
d['x']; show()
d['z']; show()

In [ ]:
df = DF(dict(a=[1,2,3],b=[4,5,6]))
df.loc[:2, ['a']]

In [ ]:
df = DF(dict(a=[{'b':2}]))
df.a.str['b']